In [ ]:
# 目标 让 llm 具有调用工具的能力，也就是所谓的 agent

In [ ]:
# 安装依赖
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [ ]:
# 相关环境变量设置
import config_loader

config_loader.load_env()

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

# 测试工具是否可以正常使用
search_results = search.invoke("北京未来一周的天气情况")
print(search_results)

# 工具可以都放到一个列表中 给模型调用
tools = [search]

In [ ]:
# 让 llm 使用工具
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_core.messages.human import HumanMessage

# 定义大模型
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 给该大模型绑定工具
llm_with_tools = llm.bind_tools(tools)

# 测试普通消息
response = llm_with_tools.invoke([HumanMessage(content='你好呀！')])
print(f'普通消息调用结果：{response.content}')
print(f"工具调用信息：{response.tool_calls}", )

In [ ]:

response = llm_with_tools.invoke("北京未来一周天气")
# 可以看到这个是空的
print(f"调用工具消息结果：{response.content}" )
# 这个不是调用工具的结果，而是大模型告诉我们要调用这个工具 所以需要创建代理
print(f"工具调用信息：{response.tool_calls}")

In [ ]:
from langgraph.prebuilt import create_react_agent

# 创建 agent
agent_executor = create_react_agent(llm, tools)

# 看看不需要工具时候的响应
response = agent_executor.invoke({"messages": [HumanMessage(content="你好~")]})

response["messages"]

In [ ]:
# 测试工具调用
response = agent_executor.invoke({"messages": [HumanMessage(content="北京今天天气情况")]})

print(type(response))
response["messages"]

In [ ]:
# 流式传输 显式实时进度
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="今天北京天气怎么样？")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# langchain-core>=0.3.37 的话 也可以添加 stream_mode="messages"
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="北京今天天气怎么样")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

In [ ]:
# agent 添加记忆
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# 就是传参的时候 多一个 checkpointer 参数
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# 设置对话线程
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字是张三")]}, config
):
    print(chunk)
    print("----")

In [ ]:
# 测试记忆
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字是什么？")]}, config
):
    print(chunk)
    print("----")